In [22]:
from FM_solution import *

## 가상의 주식 테이블 생성하기
- 시점 $T$ 는 1부터 1000까지 생성
- 개별 종목의 갯수는 400개로 설정
- 난수 발생으로 임의로 정하였다.
--------------------------------------------
## Notice : 데이터 포맷 그 이상의 의미를 가지지 못한다.

In [23]:
code_df = pd.read_csv('industrial_code.csv')
code_df.rename(columns={'종목코드':'code'},inplace=True)
code_df = code_df[:401]
num_of_stock = 400
T = 1000
df = pd.DataFrame(columns=[i for i in range(1,T+1)])

for i in df.columns:
    df[i] = [0 for i in range(num_of_stock + 1)]
    
df.index = list(code_df['code'])
df.index = [str(i) for i in df.index]

for i in range(len(df)):
    df.iloc[i] = [(np.random.randint(10))*(1+np.random.rand()) for _ in range(1000)]
    
df.index = [str(i) for i in df.index]
df = df.append(pd.Series([1 for _ in range(T)], index=df.columns), ignore_index=True) 
df.rename(index={df.index[-1]:'bias'},inplace=True)
df.rename(index={df.index[-2]:'market'},inplace=True)
individual_stock_df = df[(df.index != 'market') & (df.index != 'bias')]
df.index = list(code_df['code'])[:-1] + ['market','bias']
display(df.tail())
print(df.shape)

,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
17810,11.185765,1.283366,1.529062,1.198311,0.000000,3.511159,7.607206,12.895776,0.000000,9.013256,...,11.144508,1.158657,3.238199,7.723267,10.479106,1.299411,5.339417,6.734724,2.494051,1.708880
11200,0.000000,4.503522,3.537300,3.917704,8.647652,0.000000,3.976924,8.597750,0.000000,6.146359,...,8.470819,3.365039,5.670205,13.739712,8.764125,2.714590,5.090126,1.556178,5.775096,0.000000
17390,8.564106,7.510456,13.912607,1.334198,9.824088,1.983492,5.663772,14.612777,8.315877,10.340444,...,15.511184,3.122100,6.946294,3.811363,6.450707,10.433710,8.878859,7.664362,11.986682,0.000000
market,12.670207,3.903404,0.000000,6.858867,6.831132,10.739613,2.251839,1.811760,5.902051,11.279814,...,17.772510,4.643741,1.775416,1.838602,9.547577,8.175112,7.570554,0.000000,13.165554,9.169573
bias,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


(402, 1000)


## 1단계 : 포트폴리오 구성시기 (portfolio formation period)
- 시장모형의 시계열회귀분석으로 개별 종목에 대한 $\beta$ 계수를 추정한다.
- 개별 종목에 대한 $\beta$ 400개 계수 측정이 완료되면, 크기 순으로 나열해서 20개씩 묶어, 포트폴리오를 형성한다.
    - 개별 종목의 갯수를 400개로 한 이유가 이와 같다.

In [24]:
testing_beta , testing_resid = first_OLS(individual_stock_df,df)

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

In [25]:
len(testing_beta) , len(testing_resid)

(400, 400)

In [26]:
beta , resid , stock , idx , testing_df = first_sorting_beta_resid(testing_beta,testing_resid,df)

In [27]:
len(beta) , len(resid) , len(stock) , len(idx)

(20, 20, 20, 20)

In [28]:
testing_df.tail()

,beta,resid,valued_stock_idx,valued_stock
15,0.655919,5.561476,"[116, 310, 127, 102, 196, 161, 365, 52, 90, 33...","[23000, 21240, 138040, 68400, 101140, 119650, ..."
16,0.652303,5.567109,"[95, 212, 304, 305, 78, 163, 182, 181, 327, 38...","[13870, 28670, 64960, 37710, 200880, 115390, 1..."
17,0.647552,5.602050,"[200, 111, 172, 254, 121, 133, 222, 193, 230, ...","[100250, 129260, 47400, 84670, 69640, 134380, ..."
18,0.639652,5.607692,"[397, 17, 84, 290, 31, 213, 377, 232, 155, 96,...","[17810, 272450, 185750, 69620, 267260, 97230, ..."
19,0.622697,5.565745,"[390, 109, 14, 314, 199, 206, 211, 357, 250, 9...","[27390, 145270, 285130, 47040, 100840, 92440, ..."


# 2단계 : 포트폴리오 $\beta$ 의 추정시기 (estimation period)

- 시장 모형의 시계열회귀분석으로 20개 포트폴리오에 대한 $\beta$ 계수를 추정한다.
- 개별 종목에 대한 $\beta_i$ 를 구하고, 포트폴리오를 형성, 시계열 평균으로 $\beta_P$ 를 추정한다.

In [29]:
testing_beta2,testing_resid2 = second_OLS(individual_stock_df,df)

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

In [30]:
len(testing_beta2) , len(testing_resid2)

(400, 400)

In [31]:
final_beta , final_resid , final_df = second_sorting_beta_resid(testing_beta2,testing_resid2,df,testing_df)

In [32]:
len(final_beta) , len(final_resid)

(20, 20)

In [33]:
final_df.tail()

,beta,resid,stock_idx,stock
15,0.675610,5.582127,"[116, 310, 127, 102, 196, 161, 365, 52, 90, 33...","[23000, 21240, 138040, 68400, 101140, 119650, ..."
16,0.659320,5.571874,"[95, 212, 304, 305, 78, 163, 182, 181, 327, 38...","[13870, 28670, 64960, 37710, 200880, 115390, 1..."
17,0.677157,5.504280,"[200, 111, 172, 254, 121, 133, 222, 193, 230, ...","[100250, 129260, 47400, 84670, 69640, 134380, ..."
18,0.663262,5.544595,"[397, 17, 84, 290, 31, 213, 377, 232, 155, 96,...","[17810, 272450, 185750, 69620, 267260, 97230, ..."
19,0.667496,5.458353,"[390, 109, 14, 314, 199, 206, 211, 357, 250, 9...","[27390, 145270, 285130, 47040, 100840, 92440, ..."


# 3단계 : 검증시기(testing period)
- 앞의 2단계에 의한여 20개의 포트폴리오의 베타 $\beta_P$ 가 추정될 것이므로 이 $\beta_P$ 와 포트폴리오에 속한 개별증권의 1개월간의 기대수익률과의 횡단면 회귀분석을 하여 식의 회귀 계수를 각각 추정한다.

In [ ]:
nth_term = 1
testing_stock_df = individual_stock_df.iloc[:,900+(10*(nth_term-1)):900+(10*nth_term)]
testing_stock_df[185:200]

In [ ]:
gamma_1 , gamma_2 , gamma_3 , gamma_4 = real_df(individual_stock_df,final_df,nth_term=1)

In [ ]:
gamma_1 , gamma_2 , gamma_3 , gamma_4

# 2단계,3단계 진행
- Time batch size 를 10으로 하고 window shift를 진행한다.
- shift를 할 때마다 $\beta_P$ 를 최신화하고, cross_sectional regression을 시행한다.
- cross_sectional regression 을 할 때마다, $\gamma_1$ , $\gamma_2$ , $\gamma_3$ , $\gamma_4$ 가 새로 생긴다.
- $\gamma_1$ , $\gamma_2$ , $\gamma_3$ , $\gamma_4$ 계수들이 시계열 $T$ 만큼 생성된다.
- $\gamma_1$ , $\gamma_2$ , $\gamma_3$ , $\gamma_4$ 의 시계열 평균값을 구하고, 실증가설을 T-test에 따라 검증한다.

# Function test

In [85]:
%%time
real_gamma_1 , real_gamma_2 , real_gamma_3 , real_gamma_4 = [],[],[],[]

for i in range(10):
    testing_beta2,testing_resid2 = second_OLS(individual_stock_df,df,i=i)
    final_beta , final_resid , final_df = second_sorting_beta_resid(testing_beta2,testing_resid2,df,testing_df)
    gamma_1 , gamma_2 , gamma_3 , gamma_4 = cross_sectional_func(individual_stock_df,final_df,nth_term=i)
    real_gamma_1.append(gamma_1)
    real_gamma_2.append(gamma_2)
    real_gamma_3.append(gamma_3)
    real_gamma_4.append(gamma_4)

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

...

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

CPU times: user 15min 21s, sys: 7.33 s, total: 15min 29s
Wall time: 3min 52s


In [86]:
len(real_gamma_1),len(real_gamma_2),len(real_gamma_3),len(real_gamma_4)

(10, 10, 10, 10)

In [92]:
gamma_1 = len([j for i in real_gamma_1 for j in i])
gamma_2 = len([j for i in real_gamma_2 for j in i])
gamma_3 = len([j for i in real_gamma_3 for j in i])
gamma_4 = len([j for i in real_gamma_4 for j in i])

In [93]:
display(Markdown("### $\gamma_1$ coef's t-test : {}".format(sp.stats.ttest_1samp(real_gamma_1,popmean=0)[1][0])))
display(Markdown("### $\gamma_2$ coef's t-test : {}".format(sp.stats.ttest_1samp(real_gamma_2,popmean=0)[1][0])))
display(Markdown("### $\gamma_3$ coef's t-test : {}".format(sp.stats.ttest_1samp(real_gamma_3,popmean=0)[1][0])))
display(Markdown("### $\gamma_4$ coef's t-test : {}".format(sp.stats.ttest_1samp(real_gamma_4,popmean=0)[1][0])))

### $\gamma_1$ coef's t-test : 6.640288168948411e-08

### $\gamma_2$ coef's t-test : 6.772718512867405e-08

### $\gamma_3$ coef's t-test : 6.91627874722162e-08

### $\gamma_4$ coef's t-test : 6.645161237939336e-08